In [ ]:
import datetime
import os

try:
    from tqdm import notebook as tqdm
except ImportError:
    tqdm = None
    
import pandas as pd
import tensorflow as tf
import numpy as np
%matplotlib inline
%load_ext tensorboard
%load_ext autoreload
%autoreload 2

import dataset_ops


pd.set_option('display.max_columns', None)  # show all columns
GPUs = tf.config.list_physical_devices('GPU')
if GPUs is None or len(GPUs) == 0:
    print("WARNING: No GPU, all there is is:")
    for device in tf.config.list_physical_devices():
        print(f'- {device}')
else:
    for gpu in GPUs:
        tf.config.experimental.set_memory_growth(gpu, True)
        print("Initialized", gpu)

dataset_manager = dataset_ops.TestsManager(dataset_dir='./h5', runs_filename='runs.hdf')
all_runs = dataset_manager.get_all_available_tests()

selected_runs = all_runs.loc[(all_runs['Test Length'] > 200) & (all_runs['Test Length'] < 20000)]
# selected_runs = selected_runs.iloc[:40]
plt = selected_runs['Test Length'].plot(kind='hist', bins=25, figsize=[10,5])
plt.tick_params(labelsize=14)
plt.set_xlim([10,18000])
plt.set_xlabel('Test Length ($l_k$)', fontsize=15)
plt.set_ylabel('Number of Tests', fontsize=15)
plt.figure.savefig('test_lengths.png')
# #selected_runs
# print(all_runs.shape, selected_runs.shape)

In [ ]:
inputs = ('SpeedFts', 'Pitch', 'Roll', 'Yaw', 'current_altitude', )
outputs= ('elev', 'ai', 'rdr', 'throttle', 'Flaps')

# max_length = selected_runs['Test Length'].max()
max_length = 18000 


tfdataset = dataset_ops.TensorflowDataset(dataset_manager)
dataset = tfdataset.get_dataset(selected_runs, batch_size=25, features=inputs+outputs, max_length=max_length)
dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)


test_dataset = dataset.enumerate().filter(lambda x,y: x % 20 == 0).map(lambda x,y: y)
validation_dataset = dataset.enumerate().filter(lambda x,y: x % 20 == 1).map(lambda x,y: y)
train_dataset = dataset.enumerate().filter(lambda x,y: x % 20 > 1).map(lambda x,y: y)

dataset.element_spec

In [ ]:
from model_helper import MaskStealingLayer

def make_model(inputs, outputs, input_length):
    n_in = len(inputs)
    n_out = len(outputs)
    n_features = n_in + n_out
    
    bias_initializer = tf.keras.initializers.Constant(np.log(0.01))
    
    signals = tf.keras.Input(shape=[input_length, n_features], name='signals')
    mask = tf.keras.Input(shape=[input_length, 1], name='mask')
    
    x = signals
    x = MaskStealingLayer(0)((x, mask))
    
    x = tf.keras.layers.Conv1D(filters=64, kernel_size=3, padding="same", name='conv_3')(x)
    # x = tf.keras.layers.MaxPool1D(pool_size=2)(x)
    x = tf.keras.layers.Conv1D(filters=64, kernel_size=5, padding="same", name='conv_5')(x)
#     x = tf.keras.layers.MaxPool1D(pool_size=2)(x)
    x = tf.keras.layers.Conv1D(filters=64, kernel_size=10, padding="same", name='conv_10')(x)
#     x = tf.keras.layers.MaxPool1D(pool_size=2)(x)
    x = tf.keras.layers.Conv1D(filters=64, kernel_size=15, padding="same", name='conv_15')(x)

    x = tf.keras.layers.Conv1D(filters=64, kernel_size=20, padding="same", name='conv_20')(x)
#     x = tf.keras.layers.MaxPool1D(pool_size=2)(x)
    
    x = tf.keras.layers.GRU(128, return_sequences=True)(x)
#     x = tf.keras.layers.LeakyReLU()(x)
    
    x = tf.keras.layers.GRU(128, return_sequences=True)(x)
#     x = tf.keras.layers.LeakyReLU()(x)
    
    x = tf.keras.layers.Dense(128)(x)
    x = tf.keras.layers.LeakyReLU()(x)
    
    x = tf.keras.layers.Dense(dataset_manager.count_states(), bias_initializer=bias_initializer, activation='softmax')(x)

    # x = tf.keras.layers.UpSampling1D(2 ** 1)(x)
    
    model = tf.keras.Model(inputs=[signals, mask], outputs=x)
    
    return model

# %%

from metrics import F1, Precision, Recall, soft_dice_loss, remove_clutter_one_sample, ClassPrecision, ClassRecall

def create_prec_recall_f1(tolerance):
    prec = Precision(name=f'prec_{tolerance}', tolerance=tolerance)
    recl = Recall(name=f'recall_{tolerance}', tolerance=tolerance)

    return [
        # ClassPrecision(),
        # ClassRecall(),
        prec,
        recl,
        F1(prec, recl),
    ]

evaluation_metrics = create_prec_recall_f1(25)
optimizer = tf.keras.optimizers.Adam(lr=3e-5)

model = make_model(inputs, outputs, max_length)
model.compile(loss=soft_dice_loss, optimizer=optimizer, metrics=evaluation_metrics)

# model.summary()

In [ ]:
epochs = 100
# epochs = 5

training_start_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir="logs/fit/" + training_start_time
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

history = model.fit(train_dataset,
                    epochs=epochs,
                    validation_data=validation_dataset,
                    callbacks=[
                        tensorboard_callback,
                        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5),
                    ])
model.save(f'models/categroical-{training_start_time}-{epochs}.h5')
tf.keras.utils.plot_model(model, show_shapes=True, to_file=f'models/categorical-{training_start_time}-{epochs}.png')
model.evaluate(test_dataset)
# history.history

In [ ]:
print(model.evaluate(test_dataset))
print(model.evaluate(validation_dataset))

In [ ]:
model.load_weights('models/categroical-20200325-225905-100.h5')
# model.load_weights('models/categroical-20200507-225905-100.h5')
import matplotlib.pyplot as plt

In [ ]:
fig, axs = plt.subplots(15, 2, figsize=(15, 8), sharex=True)
N = 600

folder_name = f'Batch/categorical_{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}'
# os.mkdir(folder_name)
print('Plotting in', folder_name)

metrics_reporting = evaluation_metrics[:-1] + [
    Precision(name='prec_15', tolerance=15),
    Recall(name='recall_15', tolerance=15),
    Precision(name='prec_5', tolerance=5),
    Recall(name='recall_5', tolerance=5),
    # ClassPrecision(),
    # ClassRecall(),
]
results = []
for set_name, data_set in (('Training', train_dataset), ('Test', test_dataset)):
    for bi, data in data_set.unbatch().batch(30).enumerate():
        ins, gt = data
        prediction = model.predict_on_batch(ins)
        mask = tf.squeeze(ins['mask'], axis=-1)
        gt = tf.squeeze(gt)

        for metric in metrics_reporting: 
            metric.reset_states()
            metric.update_state(gt, prediction)
        
        prediction = tf.math.argmax(prediction, axis=-1)
        no_clutter = tf.map_fn(remove_clutter_one_sample, prediction)
        gt = tf.math.argmax(gt, axis=-1)
        
        run_length = tf.math.minimum(tf.argmin(mask, axis=-1), N)
        
        results.append(
            [set_name] + [float(metric.result()) for metric in metrics_reporting]  
        )
        
        # for prednc, truth, idx, ax in zip(no_clutter, gt, run_length, axs.reshape(-1)):
        #     if idx == 0:
        #         idx = N
        #     truth, prednc = truth[:idx], prednc[:idx]
        # 
        #     concat = tf.stack((prednc, truth), axis=0)
        #     ax.imshow(concat, aspect='auto', interpolation='nearest')
        #     ax.set_yticklabels(['', '$\\hat{O}$', '$O$'])
        #     ax.set_xlim([1, N])
        # 
        # plt.tight_layout()
        # fig.savefig(f'{folder_name}/{set_name}_{bi}.png')
        # for ax in axs.reshape(-1): ax.clear()
plt.close()
columns = ['Dataset'] + [metric.name for metric in metrics_reporting] 
results = pd.DataFrame(results, columns=columns)
pd.options.display.float_format = '{:,.2f}'.format

# results['class_precision'] *= 100
# results['class_recall'] *= 100
# results['F1'] = 2*results['class_precision']*results['class_recall'] / (results['class_precision']+results['class_recall'])

results

In [ ]:
df = results.groupby('Dataset').aggregate('mean')*100
for _tau in [5, 15, 25]:
    p = df[f'prec_{_tau}']
    r = df[f'recall_{_tau}']
    f1 = 2 * p * r / (p + r)
    df.insert(df.columns.to_list().index(f'recall_{_tau}') + 1, f"F1_{_tau}", f1)

df

In [ ]:
from sklearn.preprocessing import minmax_scale
# plt.plot(train_dataset[0]['signals'])
i = [*train_dataset.unbatch().take(1).as_numpy_iterator()][0][0]

In [ ]:
inp = model.input
outputs = [layer.output for layer in model.layers[3:]]
functors = [tf.keras.backend.function([inp], [output]) for output in outputs]

In [ ]:
inputs=[*train_dataset.unbatch().batch(1).take(1).as_numpy_iterator()]
layer_outs = [func(inputs) for func in functors]
layer_outs
# conv1 = functors[0]
# conv1()

In [ ]:
plt.plot(minmax_scale(layer_outs[9][0][0][150:900,-4]))
plt.plot(minmax_scale(layer_outs[9][0][0][150:900,-3]) + 1)
plt.plot(minmax_scale(layer_outs[9][0][0][150:900,-5]) + 2)
plt.plot(minmax_scale(layer_outs[9][0][0][150:900,-8]) + 3)
plt.axis('off')
plt.savefig('signal-out.png', bbox_inches='tight')

In [ ]:
[*enumerate(model.layers[3:])]

In [ ]:
plt.plot(minmax_scale(i['signals'][150:900,-4]))
plt.plot(minmax_scale(i['signals'][150:900,-3]) + 1)
plt.plot(minmax_scale(i['signals'][150:900,-5]) + 2)
plt.plot(minmax_scale(i['signals'][150:900,-8]) + 3)
plt.axis('off')
plt.savefig('signal.png', bbox_inches='tight')

In [ ]:
model.summary()